## Collect all item metadata and output tables linked by item id

Tables: metadata, authors, funding, tags, categories, custom fields



## Import libraries

In [1]:
import json
import requests
import pandas as pd
import csv
import datetime
import numpy as np

## Set token, admin id, and base URL

In [5]:
#Set the token in the header.
text_file = open("team-token.txt", "r") #Paste your token in a text file and save it where this notebook is
TOKEN = text_file.read()
TOKEN.strip() #removes any hidden spaces
text_file.close()


api_call_headers = {'Authorization': 'token ' + TOKEN}

#Set the base URL
BASE_URL = 'https://api.figshare.com/v2' #Change this if you want the production environment

#Set file name descriptor
descriptor = 'metadata'


## Retrieve Metadata
1. Get basic metadata by impersonating accounts (need a list of account ids with the items they own)
2. Get a list of all public metadata and get a list of ids
3. Subtract public articles from the list of private metadata ids (separating only draft items or fully embargoed items)
4. Use your choice of list of item ids to retrieve all metadata fields for each article
5. Convert the resulting JSON to a dataframe
6. Save the dataframe to CSV or Excel

In [6]:
private_items = []
for i in range(1,2):
    item = json.loads(requests.get(BASE_URL + '/account/institution/articles?page_size=1000&page={}'.format(i), headers=api_call_headers).content)
    private_items.extend(item)
print(len(private_items),'metadata records collected')


194 metadata records collected


In [7]:
#Keep records that are either public or fully embargoed, i.e. not drafts of never published records
published_items = []
for item in private_items:
    if item['published_date'] != None: #if a record has a published date
           published_items.append(item)
            
print(len(published_items), "records kept,",len(private_items) - len(published_items),"records removed")

37 records kept, 157 records removed


In [46]:
#save the json
with open('published_items-'+str(datetime.datetime.now().strftime("%Y-%m-%d"))+'.json', 'w') as f:
    json.dump(published_items, f)

## Collect metadata and views for each item
This also sets up dictionaries for certain metadata elements that will become their own tables 

In [8]:
item_metadata = []
author_metadata = []
funding_metadata = []
categories_metadata = []
tags_data = []
files_metadata = []
unpublished_count = 0

for item in published_items:
    m = requests.get(BASE_URL + '/articles/' + str(item['id']), headers=api_call_headers)
    metadata=json.loads(m.text)
    
    if m.status_code == 200: #if the record is not a previously published and then unpublished record

        views = json.loads(requests.get('https://stats.figshare.com/total/views/article/' + str(item['id']), headers=api_call_headers).content)
        #Add views to the full metadata record and add to the main list
        metadata['views'] = views['totals']
        item_metadata.append(metadata)

        #Add item id to each set of content for individual tables
        authors = metadata['authors']
        for a in authors:
            a['item_id'] = item['id']
            author_metadata.append(a)

        funding = metadata['funding_list']
        for f in funding:
            f['item_id'] = item['id']
            funding_metadata.append(f)
    
        cats = metadata['categories']
        for c in cats:
            c['item_id'] = item['id']
            categories_metadata.append(c)
    
        for t in metadata['tags']: #tags are a list so its a bit different
            tags = {}
            tags['item_id'] = item['id']
            tags['name'] = t
            tags_data.append(tags)

        if metadata['is_embargoed'] == 0: #If the record is not embargoed
            if len(metadata['files']) > 0: #If the record is not 'metadata only'
                files = metadata['files']
                for f in files:
                    f['item_id'] = item['id']
                    files_metadata.append(f)
    else:
        unpublished_count += 1
    
    
print('Full metadata for',len(item_metadata),'items retrieved.', unpublished_count,'items had a published date but are drafts')


Full metadata for 34 items retrieved. 3 items had a published date but are drafts


### Create separate tables

In [72]:
#save the json. Change the file name to represent the list of ids you used.
with open('full_records-'+str(datetime.datetime.now().strftime("%Y-%m-%d"))+'.json', 'w') as f:
    json.dump(item_metadata, f)

In [9]:
#Create a dataframe from the JSON formatted data
df = pd.DataFrame(item_metadata)
print('Done!')

Done!


## Format the spreadsheet

### Split out the dates

In [10]:
#The dates are all contained within one column called 'timeline'. Flatten that column and associate the values
#with the proper article id in a new dataframe

temp_date_list = []

for item in item_metadata:
    dateitem = item['timeline']
    dateitem['id'] = item['id']
    temp_date_list.append(dateitem)

df_dates = pd.json_normalize(
    temp_date_list 
)

df_dates.replace(df_dates.replace(r'^\s*$', 'null', regex=True, inplace = True)) #Replace blank cells with 'null'

#Merge the dataframes
df_formatted = df.merge(df_dates, how='outer', on='id')

print("Dates split out and merged")

Dates split out and merged


### Add Group names
This retrieves a list of Groups and then formats the dataframe so that each group has id of its parent Group. The top level group has itself as the parent. The group names are then added to the main dataframe.

In [11]:
#Get list of groups. 
s=requests.get(BASE_URL + '/account/institution/groups', headers=api_call_headers)
groups=json.loads(s.text)

#Create a dataframe of groups
df_groups = pd.json_normalize(groups)

df_groups_parent = df_groups[['id','name']] #Create reference dataframe
df_groups = df_groups.rename(columns={'id': 'group_id','name': 'group_name'}) #Rename id col in main dataframe
df_groups_parent = df_groups_parent.rename(columns={'name': 'parent_group_name'}) #Rename name col in reference dataframe

df_groups = df_groups.sort_values(by=['parent_id'])
top_group_id = df_groups.iloc[0]['group_id'] #Store the group id for top group 

df_groups.loc[df_groups['parent_id'] == 0, 'parent_id'] = top_group_id #For top level group, replace the zero value parent id with top level group id

df_groups = df_groups.merge(df_groups_parent, how='inner',left_on=['parent_id'], right_on=['id']) #Add parent group name

df_groups = df_groups[['group_id','group_name','parent_group_name']] #Pare down to needed columns


#Merge the dataframes 
df_formatted = df_formatted.merge(df_groups, how='inner', on='group_id') #If you use 'outer' it will include a blank record for each group with no records

print("Names for",len(df_groups),"different groups were added to the metadata records")

Names for 5 different groups were added to the metadata records


### Split out custom fields
This creates new columns for each custom field.

If different groups have different custom metadata, check the output carefully to make sure things mapped properly

In [12]:
#The custom fields are all contained within one column called 'custom_fields'. Flatten that column and associate the values
#with the proper article id in a new dataframe
custom = pd.json_normalize(
    item_metadata, 
    record_path =['custom_fields'], 
    meta=['id']
)

if len(custom) > 0:
    #This reshapes the data so that metadata field names are columns and each row is an id.
    custom = custom.pivot(index="id", columns="name", values="value")
    
    #Merge the dataframes so that all the custom fields are visible along with all the other metadata
    df_formatted = df_formatted.merge(custom, how='outer', on='id') #Outer merge keeps records that have no custom metadata.
    print('Custom fields split out and merged')
else:
    print('No custom fields to split out')

Custom fields split out and merged


### Remove the columns that are now separate tables or are slit to new columns

In [13]:
df_formatted = df_formatted.drop(columns=['files', 'authors','funding','funding_list','tags','categories','timeline','custom_fields'])


### Fill in blank cells and rename the id column

In [14]:
df_formatted.replace(df_formatted.replace(r'^\s*$', 'null', regex=True, inplace = True)) #Replace blank cells with 'null'
#df_formatted.embargo_date.fillna(value='null', inplace=True)
df_formatted.fillna(value='null', inplace=True)
df_formatted = df_formatted.rename(columns={'id': 'item_id'}) #Rename id col in main dataframe


## Create all the other dataframes for saving later

In [ ]:
authordf = pd.json_normalize(author_metadata)
authordf.replace(authordf.replace(r'^\s*$', 'null', regex=True, inplace = True)) #Replace blank cells with 'null'

funderdf = pd.json_normalize(funding_metadata)
funderdf.replace(funderdf.replace(r'^\s*$', 'null', regex=True, inplace = True)) #Replace blank cells with 'null'

categorydf = pd.json_normalize(categories_metadata)

tagdf = pd.DataFrame(tags_data, columns=["item_id","name"], index=None)

filesdf = pd.json_normalize(files_metadata)


## Get some other stats
This requires separate credentials that you need to request from Support

In [18]:
import base64

In [29]:
#Set the institution string, Basic authorization string, and base URL

INST = 'team'

text_file = open("team-stats-token.txt", "r")
auth = text_file.read()
auth.strip() #removes any hidden spaces
text_file.close()

#Set the base URL
BASE_URL2 = 'https://stats.figshare.com/'

In [20]:
#Encode the username and password 
sample_string = auth
sample_string_bytes = sample_string.encode("ascii")

base64_bytes = base64.b64encode(sample_string_bytes)
base64_string = base64_bytes.decode("ascii")

api_call_headers2 = {'Authorization': 'Basic ' + base64_string}

In [83]:
#List of public datasets:
item_ids = df_formatted['item_id'].values.tolist()
print(len(item_ids))

34


### Get views over time

In [79]:
#Get the views each month
df_timeline = pd.DataFrame(columns=['item_id','index',0]) #Set up a dataframe to accept values

for item in item_ids:
    URL = BASE_URL2 + INST + '/timeline/month/views/article/' + str(item)# + '?start_date=2018-01-01&end_date=2023-05-01'
    r = requests.get(URL, headers=api_call_headers2)
    response = r.content.decode('utf-8')
    response_dict = json.loads(response)
    if response_dict['timeline'] != None: #If there are some views to record
        #Format the JSON in the response
        df1 = pd.json_normalize(response_dict['timeline'])
        df = df1.T
        df.reset_index(inplace=True)
        df['item_id'] = item['id']
        df_timeline = df_timeline.append(df) #append data to the dataframe
    
#Rename columns 
df_timeline = df_timeline.rename(columns = {'index':'date', 0:'value'})    
    

In [81]:
save_file = df_timeline.to_csv('timeline.csv',encoding='utf-8', index=None)

## Save all dataframes to one Excel workbook

In [90]:
with pd.ExcelWriter("metadata.xlsx") as writer:
   
    # use to_excel function and specify the sheet_name and index
    # to store the dataframe in specified sheet
    df_formatted.to_excel(writer, sheet_name="main", index=False)
    authordf.to_excel(writer, sheet_name="authors", index=False)
    funderdf.to_excel(writer, sheet_name="funding", index=False)
    categorydf.to_excel(writer, sheet_name="categories", index=False)
    tagdf.to_excel(writer, sheet_name="tags", index=False)
    filesdf.to_excel(writer, sheet_name="files", index=False)
    df_timeline.to_excel(writer, sheet_name="timeline", index=False)

## Save to separate CSVs if you want

In [16]:

save_file = authordf.to_csv(descriptor + '-authors.csv',encoding='utf-8', index=None)

save_file = funderdf.to_csv(descriptor + '-funding.csv',encoding='utf-8', index=None)

save_file = categorydf.to_csv(descriptor + '-categories.csv',encoding='utf-8', index=None)

save_file = tagdf.to_csv(descriptor + '-tags.csv',encoding='utf-8', index=None)

save_file = filesdf.to_csv(descriptor + '-files.csv',encoding='utf-8', index=None)

save_file = df_formatted.to_csv(descriptor + '-main.csv',encoding='utf-8', index=None)

save_file = df_timeline.to_csv(descriptor + '-timeline.csv',encoding='utf-8', index=None)


In [10]:
#If you need to open up all the CSVs again
authordf = pd.read_csv('metadata-authors.csv')
funderdf = pd.read_csv('metadata-funding.csv')
categorydf = pd.read_csv('metadata-categories.csv')
tagdf = pd.read_csv('metadata-tags.csv')
filesdf = pd.read_csv('metadata-files.csv')
df_formatted = pd.read_csv('metadata-main.csv')


### Get views by geography - Not sure how to go about this

In [85]:
test = item_ids[15]

In [87]:
test

10766300

In [88]:
URL = BASE_URL2 + INST + '/breakdown/year/views/article/' + str(test)# + '?start_date=2018-01-01&end_date=2023-05-01'
r = requests.get(URL, headers=api_call_headers2)
response = r.content.decode('utf-8')
response_dict = json.loads(response)
#if response_dict['timeline'] != None: #If there are some views to record
##Format the JSON in the response
#df1 = pd.json_normalize(response_dict['timeline'])
#df = df1.T
#df.reset_index(inplace=True)
#df['item_id'] = item['id']
#df_timeline = df_timeline.append(df) #append data to the dataframe

In [89]:
response_dict

{'breakdown': {'2019': {'Germany': {'total': 13, 'Unknown': 13},
   'United States': {'total': 13, 'Unknown': 9, 'Ashburn': 3, 'Seattle': 1},
   'United Kingdom': {'total': 4, 'Chiswick': 3, 'New Cross': 1},
   'Russia': {'total': 3, 'Moscow': 3},
   'Australia': {'total': 2, 'Doncaster': 1, 'Melbourne': 1},
   'Canada': {'total': 1, 'Calgary': 1},
   'Romania': {'total': 1, 'Unknown': 1}},
  '2020': {'United States': {'total': 151,
    'Unknown': 81,
    'Ashburn': 44,
    'Los Angeles': 11,
    'Snohomish': 7,
    'Boydton': 3,
    'San Jose': 2,
    'Encino': 1,
    'Old Bridge': 1,
    'San Diego': 1},
   'Germany': {'total': 18, 'Unknown': 10, 'Nuremberg': 7, 'Saarbrücken': 1},
   'China': {'total': 15, 'Unknown': 13, 'Beijing': 1, 'Wuhan': 1},
   'France': {'total': 11, 'Unknown': 11},
   'Russia': {'total': 8, 'Unknown': 8},
   'Netherlands': {'total': 7, 'Unknown': 5, 'Amsterdam': 1, 'Enschede': 1},
   'India': {'total': 4, 'Pune': 4},
   'Japan': {'total': 4, 'Tokyo': 3, 'Unkn

In [ ]:
#Get the views for each article by year
#df_timeline = pd.DataFrame(columns=['item_id','index',0]) #Set up a dataframe to accept values

for item in test:
    URL = BASE_URL2 + INST + '/breakdown/year/views/article/' + str(item)# + '?start_date=2018-01-01&end_date=2023-05-01'
    r = requests.get(URL, headers=api_call_headers2)
    response = r.content.decode('utf-8')
    response_dict = json.loads(response)
    if response_dict['timeline'] != None: #If there are some views to record
        #Format the JSON in the response
        df1 = pd.json_normalize(response_dict['timeline'])
        df = df1.T
        df.reset_index(inplace=True)
        df['item_id'] = item['id']
        df_timeline = df_timeline.append(df) #append data to the dataframe
    
#Rename columns 
#df_timeline = df_timeline.rename(columns = {'index':'date', 0:'value'})    

## Adding CSVs to DataStudio

All the CSVs except the main metadata one seem to upload fine. The main metadata one has to be converted to a google sheet for some reason

Or you can upload the metadata.xlsx and connect each sheet within

In [91]:
#Or save an Excel file of all the metadata. Change the file name if necessary to match dates.
#save_file = df_formatted.to_excel('all-records-'+str(datetime.datetime.now().strftime("%Y-%m-%d"))+'.xlsx')